In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Std libs
from datetime import datetime

## Non-std libs
import matplotlib.pyplot as plt

## Local modules
from scn_rrd import librenms_meta_utils, librenms_rrd_utils, rrd_utils, plot_utils

In [ ]:
meta = librenms_meta_utils.read_meta()
devices_info = meta[['hostname', 'location']].drop_duplicates()
hostname_to_location = {
    hostname : location
    for (_, (hostname, location)) in devices_info.iterrows()
}

In [ ]:
# Hostnames gathered manually since there are only a few
cell_site_hostnames_to_pretty_names = {
    'FCS-EPC': 'FCS Cell Site EPC',
    'Franklin-EPC': 'Franklin HS Cell Site EPC',
    'OCC-EPC': 'Oromo CC Cell Site EPC',
    'Garfield-EPC': 'Garfield HS Cell Site EPC',
}

user_site_hostnames_to_pretty_names = {
    'Southend-CPEGateway-Omni': 'LIHI Southend CPE',
}

wifi_site_hostnames_to_pretty_names = {
    'CD-BackhaulGateway-hEXS': 'Nickelsville CD Lumen Backhaul',
    'Northlake-BackhaulGateway-hEXS': 'Nickelsville Northlake Lumen Backhaul',
}

In [ ]:
cell_site_dfs = librenms_rrd_utils.read_rrds(
    cell_site_hostnames_to_pretty_names.keys(),
    librenms_rrd_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.avail_daily,
    '-90days'
)

user_site_dfs = librenms_rrd_utils.read_rrds(
    user_site_hostnames_to_pretty_names.keys(),
    librenms_rrd_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.avail_daily,
    '-90days'
)

wifi_site_dfs = librenms_rrd_utils.read_rrds(
    wifi_site_hostnames_to_pretty_names.keys(),
    librenms_rrd_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.avail_daily,
    '-90days'
)

In [ ]:
# Cell Site Plot
(fig, ax) = plt.subplots(figsize=(12,6))
for (host, df) in cell_site_dfs.items():
    location = hostname_to_location[host]
    color = plot_utils.LOC_TO_COLOR[location]
    ax.plot(df['time'], df['availability'], label=cell_site_hostnames_to_pretty_names[host], color=color)
    threshold = 95
    ax.fill_between(df['time'], 0, 1, where=df['availability'] < threshold, 
                   color=color, alpha=0.2, transform=ax.get_xaxis_transform())
ax.set_title('Cell Site Availability')
ax.set_ylabel('Percentage')
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylim(bottom=90, top=101)
ax.legend()


None # Hide stdout output of the above line

In [ ]:
# CPE User Site Plot
(fig, ax) = plt.subplots(figsize=(12,6))
for (host, df) in user_site_dfs.items():
    location = hostname_to_location[host]
    color = plot_utils.LOC_TO_COLOR[location]
    ax.plot(df['time'], df['availability'], label=user_site_hostnames_to_pretty_names[host], color=color)
    threshold = 95
    ax.fill_between(df['time'], 0, 1, where=df['availability'] < threshold, 
                    color=color, alpha=0.2, transform=ax.get_xaxis_transform())
ax.set_title('CPE User Site Availability')
ax.set_ylabel('Percentage')
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylim(bottom=90, top=101)
ax.legend()
None # Hide stdout output of the above line

In [ ]:
# Wifi Site Plot
(fig, ax) = plt.subplots(figsize=(12,6))
for (host, df) in wifi_site_dfs.items():
    location = hostname_to_location[host]
    color = plot_utils.LOC_TO_COLOR[location]
    ax.plot(df['time'], df['availability'], label=wifi_site_hostnames_to_pretty_names[host], color=color)
    threshold = 95
    ax.fill_between(df['time'], 0, 1, where=df['availability'] < threshold, 
                    color=color, alpha=0.2, transform=ax.get_xaxis_transform())
ax.set_title('Wifi Site Availability')
ax.set_ylabel('Percentage')
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylim(bottom=90, top=101)
ax.legend()
None # Hide stdout output of the above line

In [ ]:
# Combined plot
# Wifi Site Plot
(fig, ax) = plt.subplots(figsize=(12,6))

# line styles (tbh this looks chaotic)
line_styles = ['dashed', 'solid', 'dotted', 'dashdot']

for (host, df) in cell_site_dfs.items():
    location = hostname_to_location[host]
    color = plot_utils.LOC_TO_COLOR[location]
    ax.plot(df['time'], df['availability'], linestyle=line_styles[0], label=cell_site_hostnames_to_pretty_names[host], color=color)
    threshold = 95
    ax.fill_between(df['time'], 0, 1, where=df['availability'] < threshold, 
                    color=color, alpha=0.2, transform=ax.get_xaxis_transform())
for (host, df) in user_site_dfs.items():
    location = hostname_to_location[host]
    color = plot_utils.LOC_TO_COLOR[location]
    ax.plot(df['time'], df['availability'], linestyle=line_styles[1], label=user_site_hostnames_to_pretty_names[host], color=color)
    threshold = 95
    ax.fill_between(df['time'], 0, 1, where=df['availability'] < threshold, 
                    color=color, alpha=0.2, transform=ax.get_xaxis_transform())
for (host, df) in wifi_site_dfs.items():
    location = hostname_to_location[host]
    color = plot_utils.LOC_TO_COLOR[location]
    ax.plot(df['time'], df['availability'], linestyle=line_styles[2], label=wifi_site_hostnames_to_pretty_names[host], color=color)
    threshold = 95
    ax.fill_between(df['time'], 0, 1, where=df['availability'] < threshold, 
                    color=color, alpha=0.2, transform=ax.get_xaxis_transform())

ax.set_title('Site Availability')
ax.set_ylabel('Percentage')
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylim(bottom=90, top=101)
ax.legend()
None # Hide stdout output of the above line